In [ ]:
import os, sys
import git

root = git.Repo('.', search_parent_directories=True).working_tree_dir
os.chdir(root)
print(f"Changed working directory to {root}")

FFMPEG = '/mmfs1/gscratch/cse/bandhav/miniconda3/envs/avhubert_gpu/bin/ffmpeg'
os.environ['PATH'] = f'{os.path.dirname(FFMPEG)}:' + os.environ['PATH']

In [ ]:
from avhubert_infer import *

In [ ]:
import IPython.display as ipd
from IPython.display import HTML

def play_video(video_path, width=200):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return ipd.display(HTML(f"""
    <video width={width} controls>
        <source src="{data_url}" type="video/mp4">
    </video>
    """))

play_video(f"{root}/data/misc/avhubert_demo_video_8s.mp4")

In [ ]:
face_predictor_path = f"{root}/data/misc/shape_predictor_68_face_landmarks.dat"
mean_face_path = f"{root}/data/misc/20words_mean_face.npy"
ckpt_path = f"{root}/data/checkpoints/base_vox_433h.pt"
cnn_detector_path = f'{root}/data/misc/mmod_human_face_detector.dat'

In [ ]:
def show_roi(video_path, cnn_detector_path=None):
    # Create a temporary file for mouth_roi_path
    with tempfile.NamedTemporaryFile(suffix='.mp4') as mouth_roi:
        mouth_roi_path = mouth_roi.name

        # Call the preprocess_video function
        preprocess_video(
            video_path, mouth_roi_path, face_predictor_path, mean_face_path,
            cnn_detector_path=cnn_detector_path
        )

        play_video(mouth_roi_path)

In [ ]:
show_roi(f"{root}/data/misc/avhubert_demo_video_8s.mp4", cnn_detector_path)

In [ ]:
show_roi(f"{root}/data/misc/avhubert_demo_video_8s.mp4")

In [ ]:
utils.import_user_module(Namespace(user_dir=work_dir))
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])

In [ ]:
features = extract_visual_features_from_video(
    origin_clip_path=f"{root}/data/misc/avhubert_demo_video_8s.mp4",
    face_predictor_path=face_predictor_path,
    mean_face_path=mean_face_path,
    models=models,
    task=task,
    cnn_detector_path=cnn_detector_path
)
print(features.shape) # [seq_len, 768]